In [ ]:
using JuMP
using Gurobi
using CSV
using DataFrames

Tx = CSV.read("Tx.csv", DataFrame, drop=1:1) |> Matrix
Ty = CSV.read("Ty.csv",DataFrame)|> Matrix;


10×90 Matrix{Float64}:
  6.0   5.0   4.0   3.0   2.0   3.0  …   7.0   8.0   9.0  10.0  11.0  12.0
 11.0  10.0   9.0   8.0   7.0   6.0     10.0   9.0   8.0   7.0   6.0   7.0
  4.0   3.0   4.0   5.0   6.0   7.0      9.0  10.0  11.0  12.0  13.0  14.0
  9.0   8.0   7.0   6.0   5.0   4.0     14.0  13.0  12.0  11.0  10.0   9.0
  6.0   5.0   6.0   7.0   8.0   9.0      7.0   8.0   9.0  10.0  11.0  12.0
 10.0   9.0   8.0   7.0   8.0   9.0  …   3.0   4.0   5.0   6.0   7.0   8.0
 16.0  15.0  14.0  13.0  12.0  11.0      7.0   6.0   5.0   4.0   3.0   2.0
 11.0  10.0   9.0   8.0   9.0  10.0      2.0   3.0   4.0   5.0   6.0   7.0
  9.0   8.0   7.0   8.0   9.0  10.0      4.0   5.0   6.0   7.0   8.0   9.0
  9.0   8.0   7.0   6.0   7.0   8.0      4.0   5.0   6.0   7.0   8.0   9.0

In [21]:
Ty

89×10 Matrix{Float64}:
 10.0   6.0   8.0   3.0  11.0   5.0  16.0  11.0   3.0  12.0
 11.0   7.0   9.0   4.0  10.0   4.0  15.0  10.0   2.0  11.0
 12.0   8.0  10.0   5.0   9.0   3.0  14.0   9.0   1.0  10.0
 13.0   9.0  11.0   6.0   8.0   2.0  13.0   8.0   2.0   9.0
 14.0  10.0  12.0   7.0   7.0   1.0  12.0   7.0   3.0   8.0
 16.0  12.0  14.0   9.0   5.0   1.0  10.0   9.0   5.0  10.0
 17.0  13.0  15.0  10.0   6.0   2.0   9.0  10.0   6.0  11.0
 18.0  14.0  16.0  11.0   7.0   3.0  10.0  11.0   7.0  12.0
  8.0   6.0   6.0   1.0  11.0   7.0  16.0  11.0   3.0  12.0
  9.0   5.0   7.0   2.0  10.0   6.0  15.0  10.0   2.0  11.0
 10.0   6.0   8.0   3.0   9.0   5.0  14.0   9.0   1.0  10.0
 12.0   8.0  10.0   5.0   7.0   3.0  12.0   7.0   1.0   8.0
 13.0   9.0  11.0   6.0   6.0   2.0  11.0   6.0   2.0   7.0
  ⋮                             ⋮                      
  7.0   7.0   7.0  12.0   4.0   8.0   3.0   2.0  10.0   1.0
  8.0   8.0   8.0  13.0   3.0   9.0   2.0   3.0  11.0   2.0
  9.0   9.0   9.0  14

In [1]:
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "TimeLimit", 120)

@variable(model, o[1:N], Bin)
@variable(model, x[1:N, 1:N, 1:N], Bin)
@variable(model, y[1:N, 1:M, 1:N], Bin)
@variable(model, z[1:N, 1:N, 1:M], Bin)
@variable(model, f[1:N] >= 0)
@variable(model, L >= 0)

@Objective(model, Min, alpha*(Cf*sum(o[i] for i in 1:M) + 
Ct * sum(y[k, i, j] for k in 1:N, i in 1:M, j in 1:N) + 
Cw * sum(sum(y[k,i,j]*Ty[i,j] + z[k,j,i]*Tz[j,i] for i in 1:M, j in 1:N) + sum(x[k, j1, j2] * Tx[j1, j2] for j1 in 1:N, j2 in 1:N) for k in 1:N)) +
(1 - alpha) * L)

# Optional additional constriant
for j in 1:N, k in 1:N
    @constriant(model, x[k,j,j] == 0)
end
# end additional constraint

for k in 1:N
    @constraint(model, sum(y[k,i,j] for i in 1:M, j in 1:N) <= 1)
end

for k in 1:N
    @constraint(model, sum(x[k,j1,j2] for j1 in 1:N, j2 in 1:N) <= (N - 1) * sum(y[k,i,j] for i in 1:M, j in 1:N))
end

for i in 1:M
    @constraint(model, sum(y[k,i,j] for k in 1:N, j in 1:N) <= N * o[i])
end

for k in 1:N, i in 1:M
    @costraint(model, sum(z[k,j,i] for j in 1:N) == sum(y[k,i,j] for j in 1:N))
end

for j2 in 1:N 
    @constraint(model, sum(sum(y[k,i,j2] for i in 1:M) + sum(x[k,j1,j2] for j1 in 1:N) for k in 1:N) == 1)
end 

for j1 in 1:N 
    @constraint(model, sum(sum(x[k,j1,j2] for j2 in 1:N) + sum(z[k,j1,i] for i in 1:M) for k in 1:N) == 1)
end 

# Now time constraints
for k in 1:N 
    @constraint(model, f[k] == sum(y[k,i,j] * Ty[i,j] for i in 1:M, j in 1:N) + sum(x[k,j1,j2] * Tx[j1,j2] for j1 in 1:N, j2 in 1:N))
end

for k in 1:N 
    @constraint(model, f[k] <= L)
end





LoadError: UndefVarError: `Gurobi` not defined